In [1]:
import time
import sys
import urx
import numpy as np
import math
import threading
import io
import socket
import struct
from PIL import Image
import cv2
import numpy
import pupil_apriltags as apriltag
import time
from scipy.spatial.transform import Rotation as R
import transforms3d

In [2]:
tag = False
tag_pose = []

In [3]:
def coord_receiver(connection,detector,camera_params,tag_size,robot,goal,home,accuracy):
    global tag,tag_pose,threads_active
    print("coord_reciever thread started")
    
    while threads_active:
        image_len = struct.unpack(
            '<L', connection.read(struct.calcsize('<L')))[0]
        # print('I am here')
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        opencvImage = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
        result = detector.detect(
            gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)

        if result:
            tag = True
            tag_pose = result[0].pose_t
            print('coords=',tag_pose)
        else:
            tag=False
            tag_pose = []
        cv2.imshow('Image', opencvImage)

        current_rob_pose = robot.getl()
        diff = np.array(current_rob_pose[:3])-np.array(goal)
        v = diff/np.max(np.abs(diff))*0.02
        robot.speedl((v[0],v[1],v[2],0,0,0),0.05,1)
        distance = np.linalg.norm(diff)

        if distance <=accuracy:
            rob.speedl((0,0,0,0,0,0),0.05,1)
            threads_active = False

        if tag and np.linalg.norm(tag_pose) < 0.2:
            print('COLLISION')
            robot.speedl((0,0,0,0,0,0),0.05,1)
            robot.movel((home[0],home[1],home[2],home[3],home[4],home[5]),0.01,0.01)
            threads_active = False
        key = cv2.waitKey(1) & 0xFF





        if key == ord('q'):
            threads_active = False
        

        


In [4]:
camera_params = (506.19083684, 508.36108854,
                 317.93111342, 243.12403806)
tag_size = 0.0375

# UDP_IP = "127.0.0.1"
# UDP_PORT = 5065
# sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# server_socket = socket.socket()
# server_socket.bind(('0.0.0.0', 8000))
# server_socket.listen(0)
# connection = server_socket.accept()[0].makefile('rb')

accuracy = 0.001

detector = apriltag.Detector(
    families='tag36h11',
    nthreads=10,
    quad_decimate=0.5,
    quad_sigma=0,
    refine_edges=1,
    decode_sharpening=0.6,
    debug=0)



FileNotFoundError: Could not find module 'c:\Users\denis\anaconda3\lib\site-packages\pupil_apriltags\lib\apriltag.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [6]:
rob = urx.Robot('192.168.88.60')
home = [-0.314,0.107,0.483,-0.013, -1.55, 0.029]
goal = [-0.414,0.107,0.483,-0.013, -1.55, 0.029]

In [8]:
rob.getl()

AttributeError: module 'collections' has no attribute 'Iterable'

Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\urx\ursecmon.py", line 291, in run
    data = self._get_data()
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\urx\ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
TimeoutError: timed out


In [7]:
threads_active=True
coord_rec_thread = threading.Thread(
target=coord_receiver,
daemon=True,
args=(connection,detector,camera_params,tag_size,rob,goal,home,accuracy)
)
coord_rec_thread.start()

coord_reciever thread started


Exception in thread Thread-4 (coord_receiver):
Traceback (most recent call last):
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\denis\AppData\Local\Temp\ipykernel_19292\746744183.py", line 29, in coord_receiver
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\urx\robot.py", line 210, in getl
    t = self.get_pose(wait, _log)
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\urx\robot.py", line 128, in get_pose
    trans = self.csys.inverse * m3d.Transform(pose)
  File "c:\Users\denis\AppData\Local\Programs\Python\Python310\lib\site-packages\math3d\transform.py", line 234, in get_inverse
    return Transform(np.linalg.inv(self._data))
  File "c:\Users\denis\AppData\Local\Programs\Pytho